# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

# Загрузка данных

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv('../data/kc_house_data.csv')
data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Предобработка полученных данных

In [4]:
data.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [5]:
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [6]:
for i in data['date']:
    if i.endswith('T000000') == False:
        print(i)

In [7]:
data['date'] = data['date'].str.replace('T000000', '')
data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [8]:
data_year = []
data_month = []
data_day = []

for i in range(data.shape[0]):
    
    data_year.append(data['date'][i][:4])
    data_month.append(data['date'][i][4:6])
    data_day.append(data['date'][i][6:])


In [9]:
data = data.drop(['date', 'id'], axis=1)
data

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [10]:
data.insert(loc = 1, column = 'year', value = data_year)

In [11]:
data.insert(loc = 1, column = 'month', value = data_month)

In [12]:
data.insert(loc = 1, column = 'day', value = data_day)

In [13]:
data.to_csv('../data/pre_kc_house_data.csv')

In [7]:
data = pd.read_csv('../data/pre_kc_house_data.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data

,price,day,month,year,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,13,10,2014,3,1.00,1180,5650,1.0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,9,12,2014,3,2.25,2570,7242,2.0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,25,2,2015,2,1.00,770,10000,1.0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,9,12,2014,4,3.00,1960,5000,1.0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,18,2,2015,3,2.00,1680,8080,1.0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,21,5,2014,3,2.50,1530,1131,3.0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,400000.0,23,2,2015,4,2.50,2310,5813,2.0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,402101.0,23,6,2014,2,0.75,1020,1350,2.0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,400000.0,16,1,2015,3,2.50,1600,2388,2.0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Распределение на обучающую и тестовую выборки

In [6]:
y = data["price"]
X = data.drop(["price"], axis=1)

In [7]:
y

0        221900.0
1        538000.0
2        180000.0
3        604000.0
4        510000.0
           ...   
21608    360000.0
21609    400000.0
21610    402101.0
21611    400000.0
21612    325000.0
Name: price, Length: 21613, dtype: float64

In [8]:
X

,day,month,year,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,13,10,2014,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,9,12,2014,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,25,2,2015,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,9,12,2014,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,18,2,2015,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,21,5,2014,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,23,2,2015,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,23,6,2014,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,16,1,2015,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [3]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((17290, 21), (17290,), (4323, 21), (4323,))

# Линейная регрессия

In [4]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [13]:
lr = LinearRegression().fit(X_train, y_train)

In [14]:
lr.coef_

array([-3.97146014e+02,  8.96550802e+02,  3.49259612e+04, -3.41730217e+04,
        4.12499582e+04,  1.08362878e+02,  1.50294637e-01, -2.76128191e+02,
        5.95749529e+05,  5.18529046e+04,  2.86444365e+04,  9.08650985e+04,
        7.32228355e+01,  3.51400425e+01, -2.92208339e+03,  2.16078798e+01,
       -5.91031610e+02,  5.84931105e+05, -1.91530661e+05,  3.58866156e+01,
       -4.41155799e-01])

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

In [16]:
y_pred = lr.predict(X_test)
y_pred

array([664109.75211319, 697230.72265098, 313078.72440029, ...,
        92653.14193954, 379216.69708499, 102408.68785483])

In [17]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 131007.90380184942
MSE: 43004489126.36222
RMSE: 207375.23749561378
MAPE: 0.24509555529963767
R^2: 0.6868738238252998


## Линейная регрессия + Ridge-регуляризация

In [13]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [14]:
parameters = {'alpha': np.arange(0, 1, 0.1)}

In [15]:
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)

D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.38395e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.30383e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [16]:
ridge_optimal.best_params_

{'alpha': 0.9}

In [24]:
ridge = Ridge(alpha=0.9).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 131015.73185950723
MSE: 43011492433.63926
RMSE: 207392.1224001511
MAPE: 0.24507009399360846
R^2: 0.6868228310365752


array([-3.97074251e+02,  8.96780054e+02,  3.48991820e+04, -3.42025036e+04,
        4.12547838e+04,  1.08385136e+02,  1.49567907e-01, -2.22784156e+02,
        5.91009020e+05,  5.20388351e+04,  2.86225402e+04,  9.09190985e+04,
        7.31908435e+01,  3.51918952e+01, -2.92531130e+03,  2.16251766e+01,
       -5.88967461e+02,  5.83028706e+05, -1.90515067e+05,  3.58698411e+01,
       -4.41653623e-01])

### стандартизировав признаки

In [66]:
ridge = Ridge(alpha=0.9).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 131004.82784530909
MSE: 43004416068.30996
RMSE: 207375.06134612707
MAPE: 0.24508566839190948
R^2: 0.6868743557788006


array([ -3425.90818889,   2788.96885956,  16338.47714125, -31978.75530077,
        31281.33695962,  79631.64264673,   6250.71937855,   -138.03614024,
        52435.63962718,  40406.94907957,  19093.25045372, 106238.50076349,
        75381.62156713,  24941.17858961, -81536.54495176,   8925.50181981,
       -31980.8681909 ,  81749.35999451, -26632.21053992,  24192.06716345,
       -12021.80697171])

In [25]:
y_pred

array([663739.89018045, 696832.060136  , 313205.88010161, ...,
        92562.34945718, 379240.03038777, 102322.55961216])

## Линейная регрессия + Lasso-регуляризация

In [26]:
lasso_optimal = GridSearchCV(Lasso(), parameters).fit(X_train, y_train)

D:\Machine_Learning\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.729e+14, tolerance: 1.824e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = 

In [27]:
lasso_optimal.best_params_

{'alpha': 0.9}

In [28]:
lasso = Lasso(alpha=0.9).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 131007.45924631208
MSE: 43004850935.16967
RMSE: 207376.10984674603
MAPE: 0.24509223204083433
R^2: 0.6868711894071104


D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.156e+14, tolerance: 2.315e+11
  model = cd_fast.enet_coordinate_descent(


array([-3.97158170e+02,  8.95148655e+02,  3.49138091e+04, -3.41718901e+04,
        4.12440277e+04,  2.80317688e+02,  1.50266239e-01, -2.65444380e+02,
        5.95618892e+05,  5.18571182e+04,  2.86416461e+04,  9.08663788e+04,
       -9.87333750e+01, -1.36805765e+02, -2.92225413e+03,  2.16076862e+01,
       -5.90918441e+02,  5.84870213e+05, -1.91445340e+05,  3.58861884e+01,
       -4.41196058e-01])

In [29]:
y_pred

array([664094.96834226, 697212.30901819, 313090.69297559, ...,
        92661.26174209, 379221.69841181, 102411.51927965])

# Полиномиальная регрессия

In [30]:
from sklearn.preprocessing import PolynomialFeatures
p = PolynomialFeatures(2)  
X_p = p.fit_transform(X) 
X_p

array([[1.0000000e+00, 1.3000000e+01, 1.0000000e+01, ..., 1.7956000e+06,
        7.5710000e+06, 3.1922500e+07],
       [1.0000000e+00, 9.0000000e+00, 1.2000000e+01, ..., 2.8561000e+06,
        1.2909910e+07, 5.8354321e+07],
       [1.0000000e+00, 2.5000000e+01, 2.0000000e+00, ..., 7.3984000e+06,
        2.1928640e+07, 6.4995844e+07],
       ...,
       [1.0000000e+00, 2.3000000e+01, 6.0000000e+00, ..., 1.0404000e+06,
        2.0471400e+06, 4.0280490e+06],
       [1.0000000e+00, 1.6000000e+01, 1.0000000e+00, ..., 1.9881000e+06,
        1.8146700e+06, 1.6563690e+06],
       [1.0000000e+00, 1.5000000e+01, 1.0000000e+01, ..., 1.0404000e+06,
        1.3841400e+06, 1.8414490e+06]])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_p, y, test_size=0.2, shuffle=False)

In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((17290, 253), (17290,), (4323, 253), (4323,))

In [33]:
lr2 = LinearRegression().fit(X_train, y_train)
lr2.coef_

array([-2.91337122e+03, -1.22473755e+06, -3.14013653e+07,  1.99747323e+04,
        3.65769952e+07, -7.82071765e+07,  4.80648963e+05, -3.90833068e+02,
       -2.17796427e+07, -2.71611886e+07,  1.31633084e+07,  2.09040203e+07,
       -2.45384485e+07, -4.50689243e+05, -5.62634545e+05,  1.60345056e+06,
        7.27933933e+04, -6.94419839e+05,  1.02745316e+08, -1.26170792e+07,
       -2.12392495e+04,  6.99447777e+02,  4.59574271e+01,  1.09873795e+02,
        4.24775833e+02,  6.97879690e+01,  6.38075161e+02,  3.10279066e+03,
       -1.10549170e-03,  2.80022557e+01,  5.33539947e+03, -4.36259485e+01,
        2.15067077e+01, -2.74742972e+02, -3.10350960e+03, -3.10331025e+03,
        9.46024619e+00,  1.26798641e-01,  5.92385591e+00, -1.64955295e+03,
        1.25749535e+03,  4.07174869e-01,  1.19555660e-02,  1.76086324e+02,
        1.46032036e+04, -8.10432390e+02,  4.36845898e+03, -1.12400859e+01,
        1.35764624e-02, -2.18819616e+03,  1.20578223e+04, -3.53691617e+02,
       -1.38930087e+03,  

In [34]:
y_pred = lr2.predict(X_test)
y_pred

array([651551.52993393, 687193.0321312 , 405968.57522964, ...,
       450887.83014679, 448550.59673309, 409000.53280258])

In [35]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lr2.score(X_test, y_test)}')

MAE: 105711.36322411867
MSE: 27905160470.71967
RMSE: 167048.3776357007
MAPE: 0.20089028187952482
R^2: 0.7968157192133354


## Полиномиальная регрессия + Ridge-регуляризация 

In [36]:
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)

In [37]:
ridge_optimal.best_params_

{'alpha': 0.1}

In [38]:
ridge = Ridge(alpha=0.1).fit(X_train, y_train)
y_pred = ridge.predict(X_test)

In [39]:
ridge.coef_

array([-1.81870295e+02, -1.19771997e+05, -1.10363325e+05, -1.38756423e+01,
        5.25411743e+04, -5.14058759e+04, -3.71132482e+04, -3.45594831e+02,
       -2.76716010e+04, -1.00873204e+03, -3.68242397e+02,  8.95629698e+03,
       -3.44484703e+04,  2.35658301e+04, -6.03870140e+04,  2.68787144e+05,
        5.02430829e+04, -5.92093252e+05,  2.78295854e+03,  2.36722817e+03,
       -2.71073053e+04,  7.37172272e+02,  4.57950306e+01, -8.63499289e+00,
       -1.87294638e+02,  1.55043767e+02,  4.92330542e+02, -8.69080232e+02,
       -9.05237900e-04, -1.77848322e+01,  4.89426242e+03, -4.85746755e+00,
        2.63740297e+01, -2.95139871e+02,  8.68517155e+02,  8.68502459e+02,
        2.67790610e+00, -7.90765417e-02,  7.29128817e+00, -1.43382072e+03,
        1.27738280e+03,  4.50152797e-01,  1.25579887e-02, -8.78978813e+02,
       -6.30488206e+02,  1.15627729e+03,  2.24959057e+02, -1.16038099e+03,
        1.37215896e-02, -3.24997588e+03,  1.02512363e+04,  3.50014955e+02,
       -3.53217487e+02, -

In [40]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {ridge.score(X_test, y_test)}')

MAE: 105384.90029541464
MSE: 27849050541.18284
RMSE: 166880.3479777737
MAPE: 0.1998845001983962
R^2: 0.7972242692981806


## Полиномиальная регрессия + Lasso-регуляризация

In [41]:
lasso_optimal = GridSearchCV(Lasso(), parameters).fit(X_train, y_train)

D:\Machine_Learning\venv\lib\site-packages\sklearn\model_selection\_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.857e+14, tolerance: 1.824e+11 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = 

In [42]:
lasso_optimal.best_params_

{'alpha': 0.9}

In [43]:
lasso = Lasso(alpha=0.9).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
print(f'R^2: {lasso.score(X_test, y_test)}')

MAE: 115182.69117439126
MSE: 35016789112.04832
RMSE: 187127.73474834862
MAPE: 0.20724540937937366
R^2: 0.7450342162104631


D:\Machine_Learning\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.374e+14, tolerance: 2.315e+11
  model = cd_fast.enet_coordinate_descent(


In [44]:
lasso.coef_

array([ 0.00000000e+00, -5.79299615e+02,  4.47260906e+03,  4.10645513e+04,
        1.92993266e+04,  1.34353457e+05,  6.24814739e+01,  1.20984445e-01,
       -2.49683385e+04,  7.28988723e+05,  4.97189347e+04, -6.82284845e+03,
        5.88419052e+04, -8.96736487e+01, -2.20575033e+01, -1.98908459e+03,
       -9.27121908e+01,  4.60122651e+02,  4.18247025e+05,  2.36164237e+04,
        2.84238969e+01,  2.79153922e-01,  4.88138552e+01,  5.49529066e+01,
       -7.78369878e-01, -1.73250964e+02,  4.15657126e+02, -6.83313680e-01,
       -1.47966420e-03, -2.79178099e+02,  3.97499018e+03, -1.85605845e+01,
       -5.96461606e+01, -1.65021204e+02,  6.92217368e-01,  1.59322121e-01,
        1.11980389e+00, -1.78302882e-01, -3.48170727e-03, -1.31863889e+01,
       -2.87057518e+00, -2.23295226e-03,  1.45666644e-02, -1.00015281e+03,
        8.19133839e+00,  8.05945697e+02, -1.82202059e+02, -8.67952571e-01,
        1.21639173e-03, -4.21496842e+02,  7.46276943e+03,  1.95588065e+01,
       -9.04804208e+02, -

# Стандартизация предикторов

In [8]:
y = data["price"]
X = data.drop(["price"], axis=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [10]:
means = X_train.mean()
stds = X_train.std()

display(means, stds)

day                 15.690226
month                6.573279
year              2014.323655
bedrooms             3.366744
bathrooms            2.062927
sqft_living       2054.035454
sqft_lot         15696.233256
floors               1.433198
waterfront           0.007808
view                 0.242568
condition            3.449682
grade                7.594910
sqft_above        1751.377964
sqft_basement      302.657490
yr_built          1967.369578
yr_renovated        89.453730
zipcode          98077.980278
lat                 47.560008
long              -122.214968
sqft_living15     1975.198612
sqft_lot15       13263.367843
dtype: float64

day                  8.626413
month                3.113018
year                 0.467884
bedrooms             0.935958
bathrooms            0.758378
sqft_living        904.136076
sqft_lot         41593.808900
floors               0.509828
waterfront           0.088020
view                 0.779260
condition            0.666525
grade                1.169304
sqft_above         806.221544
sqft_basement      450.000931
yr_built            27.906944
yr_renovated       412.965024
zipcode             54.120538
lat                  0.139765
long                 0.139055
sqft_living15      673.869689
sqft_lot15       27254.651043
dtype: float64

In [11]:
X_train = (X_train - means) / stds
X_train.describe()

,day,month,year,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,...,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04,1.729000e+04
mean,7.438302e-17,-7.222550e-17,-1.389746e-13,2.270531e-16,-1.315059e-17,6.914333e-17,-6.164338e-18,4.725993e-17,1.972588e-17,-4.931471e-18,...,-1.027390e-17,5.198592e-17,1.869849e-17,-2.838061e-15,4.315037e-17,1.291256e-13,-5.256640e-15,1.485893e-14,-9.143769e-17,-6.986250e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.702936e+00,-1.790314e+00,-6.917430e-01,-3.597108e+00,-2.720184e+00,-1.862591e+00,-3.648676e-01,-8.496944e-01,-8.870722e-02,-3.112800e-01,...,-3.929612e+00,-1.713397e+00,-6.725708e-01,-2.414079e+00,-2.166133e-01,-1.422386e+00,-2.891341e+00,-2.186423e+00,-2.339026e+00,-4.624300e-01
25%,-8.914743e-01,-8.266188e-01,-6.917430e-01,-3.918377e-01,-7.422774e-01,-7.012611e-01,-2.457874e-01,-8.496944e-01,-8.870722e-02,-3.112800e-01,...,-5.087732e-01,-7.087109e-01,-6.725708e-01,-6.224106e-01,-2.166133e-01,-8.311129e-01,-6.811989e-01,-8.056702e-01,-7.200185e-01,-2.870012e-01
50%,3.591000e-02,-1.841555e-01,-6.917430e-01,-3.918377e-01,-8.297522e-02,-1.703676e-01,-1.871488e-01,-8.496944e-01,-8.870722e-02,-3.112800e-01,...,-5.087732e-01,-2.745870e-01,-6.725708e-01,5.842353e-02,-2.166133e-01,-2.398401e-01,9.188587e-02,-1.116979e-01,-2.154699e-01,-1.991538e-01
75%,8.473713e-01,7.795395e-01,1.445540e+00,6.765859e-01,5.763270e-01,4.932494e-01,-1.110137e-01,1.111750e+00,-8.870722e-02,-3.112800e-01,...,3.464366e-01,4.820288e-01,6.607598e-01,8.109244e-01,-2.166133e-01,7.394554e-01,8.520919e-01,6.326143e-01,5.413530e-01,-1.071970e-01
max,1.774756e+00,1.743235e+00,1.445540e+00,3.166087e+01,7.828651e+00,1.270380e+01,3.932467e+01,4.053916e+00,1.127239e+01,4.821797e+00,...,4.622486e+00,9.499401e+00,1.003852e+01,1.706759e+00,4.662735e+00,2.236115e+00,1.556848e+00,6.472049e+00,6.135906e+00,3.147854e+01


In [12]:
X_test = (X_test - means) / stds
X_test.describe()

,day,month,year,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,...,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000,4323.000000
mean,-0.001176,0.001836,-0.007496,0.021889,0.341690,0.143020,-0.070829,0.599270,-0.015122,-0.053023,...,0.264932,0.229523,-0.123860,0.651311,-0.061155,-0.003739,0.001607,0.038524,0.084236,-0.090786
std,1.005119,1.003786,0.997229,0.968008,1.031231,1.069274,0.977055,1.149786,0.911477,0.912833,...,0.998424,1.110424,0.907900,1.095508,0.853031,0.941874,0.956377,1.061805,1.080282,1.005896
min,-1.702936,-1.790314,-0.691743,-3.597108,-2.720184,-1.951073,-0.362103,-0.849694,-0.088707,-0.311280,...,-5.640032,-1.812626,-0.672571,-2.414079,-0.216613,-1.422386,-2.680988,-2.085743,-2.011069,-0.462760
25%,-0.891474,-0.826619,-0.691743,-0.391838,-0.412626,-0.665868,-0.285998,-0.849694,-0.088707,-0.311280,...,-0.508773,-0.624367,-0.672571,-0.120743,-0.216613,-0.831113,-0.519499,-0.856010,-0.734858,-0.339882
50%,0.035910,0.137076,-0.691743,-0.391838,0.576327,-0.059765,-0.233117,1.111750,-0.088707,-0.311280,...,0.346437,-0.076130,-0.672571,1.240925,-0.216613,-0.239840,0.052892,-0.108102,-0.156111,-0.266500
75%,0.847371,0.779540,1.445540,0.676586,0.576327,0.747636,-0.157241,1.111750,-0.088707,-0.311280,...,1.201646,0.953363,0.257427,1.491759,-0.216613,0.720978,0.811667,0.718911,0.719429,-0.156427
max,1.774756,1.743235,1.445540,7.087127,5.850744,6.598525,27.626654,4.053916,11.272388,4.821797,...,4.622486,7.775310,5.571861,1.706759,4.662735,2.236115,1.556848,6.119670,6.284303,30.999063


# Реализация. Линейная регрессия + Ridge-регуляризация

In [62]:
class Ridge_Regression:
    
    def __init__(self, rate = 0.01, iters = 5000, alpha = 0.9):
        
        self.rate = rate
        self.iters = iters
        self.alpha = alpha
    
    def add_ones(self, x):
        
        return np.concatenate((np.ones((len(x), 1)), x), axis=1)
    
    def ridge_error(self, x, y, w):
        
        return np.sum((y - np.dot(x, w)) ** 2) / x.shape[0] + np.sum(w ** 2) * self.alpha / x.shape[0]
    
    def gradient(self, x, y, w):
        
        return 2 / x.shape[0] * np.dot(y - np.dot(x, w), -x) + 2 / x.shape[0] * self.alpha * w
         
    def coef(self):
        
        return self.w[:-1]
    
    def intercept(self):
        
        return self.w[-1]
    
    def predict(self, x):
        
        return np.dot(self.add_ones(x), self.w) 
    
    def fit(self, x, y):
        
        delta_error = np.inf
        eps = 1e-4
        X = self.add_ones(x)
        w = np.zeros(X.shape[1])
        i = 0
        
        while delta_error > eps and i <= self.iters:
            
            error = self.ridge_error(X, y, w)
            w -= self.rate * self.gradient(X, y, w)
            delta_error = error - self.ridge_error(X, y, w)
            i += 1
            
            print(f'Итерация: {i}\n')
            print(f'Веса: {w}\n')
            print(f'Delta_error: {delta_error}\n')
            print("--------------------------------------------------------")
        
        self.w = w
        

In [63]:
ridge = Ridge_Regression()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

Итерация: 1

Веса: [10664.94986813  -114.56991439   -41.95542997    11.57944563
  2205.10491728  3828.84024226  5133.10249088   679.7138209
  2059.01818649  2020.99856843  2960.65647896   334.55096966
  4857.40963528  4429.31241499  2377.8085258    273.37798824
   981.91031808  -325.96372359  2253.01610173    99.89817619
  4393.99811584   581.55100363]

Delta_error: 24453559970.436707

--------------------------------------------------------
Итерация: 2

Веса: [21116.589636    -222.81594938   -91.04972033    39.73995113
  4129.18685321  7245.63353767  9790.70174809  1261.33939803
  3864.26151407  3929.6308581   5710.76683896   709.83894839
  9281.33792747  8424.88818101  4577.32453813   352.13519293
  1912.78724769  -555.98015127  4432.53547931    79.54911975
  8369.90301383  1062.0053429 ]

Delta_error: 21710492002.957703

--------------------------------------------------------
Итерация: 3

Веса: [31359.18572769  -325.27995473  -146.24511151    82.53447818
  5801.43692035 10293.33044

## Реализация. Метрики качества

In [64]:
def mae_error(y_pred, y):
    return np.sum(abs(y-y_pred))/y_pred.shape[0]

def mse_error(y_pred, y):
    return np.sum((y-y_pred)**2)/y_pred.shape[0]

def rmse_error(y_pred, y):
    return np.sqrt(np.sum((y-y_pred)**2)/y_pred.shape[0])

def mape_error(y_pred, y):
     return np.sum(abs((y-y_pred)/y))/y_pred.shape[0]

def r2_coef(y_pred, y):
    return 1 - (mse_error(y_pred, y)/(np.sum((y-np.mean(y))**2)/y_pred.shape[0]))

In [65]:
print(f'MAE:{mae_error(y_pred, y_test)}')
print(f'MSE:{mse_error(y_pred, y_test)}')
print(f'RMSE:{rmse_error(y_pred, y_test)}')
print(f'MAPE:{mape_error(y_pred, y_test)}')
print(f'R^2:{r2_coef(y_pred, y_test)}')

MAE:131008.07253420708
MSE:43006145047.58348
RMSE:207379.230029392
MAPE:0.2450910892736776
R^2:0.6868617666589297
